**CS 513 - Theory And Data Cleaning Final Project - Phase 2 ICV SQLITE Notebook**

Team ID: Team 150

Sushma Ponna - ponna2@illinois.edu

Michael Inoue - mwinoue2@illinois.edu

Sanjeev Datta - sanjeev6@illinois.edu

**Requirements**: In Google Drive, create a folder 'CS513' under 'My Drive'. Within this folder, place Menu.csv, Dish.csv, MenuItem.csv, MenuPage.csv from the NYPL menus dataset and Menu_cleaned.csv, Dish_cleaned.csv, Menu_item_cleaned.csv, and menu_join_menu_page.csv from the [cleaned datset.](https://github.com/sushma9/CS513FinalProjectRepo/tree/main/Data_Cleaned)

In [3]:
!pip install pysqlite3
import sqlite3
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
from sqlalchemy import create_engine
import os

Mounted at /content/drive/


In [4]:
project_db_url = '/content/drive/My Drive/CS513/project.db'

conn = sqlite3.connect(project_db_url)

menu_csv = '/content/drive/My Drive/CS513/Menu.csv'
dish_csv = '/content/drive/My Drive/CS513/Dish.csv'
menuitem_csv = '/content/drive/My Drive/CS513/MenuItem.csv'
menupage_csv = '/content/drive/My Drive/CS513/MenuPage.csv'

df = pd.read_csv(menu_csv)
try:
  df.to_sql('Menu', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(dish_csv)
try:
  df.to_sql('Dish', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(menuitem_csv)
try:
  df.to_sql('MenuItem', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(menupage_csv)
try:
  df.to_sql('MenuPage', conn, index=False)
except Exception as e:
  print(e)


Table 'Menu' already exists.
Table 'Dish' already exists.
Table 'MenuItem' already exists.
Table 'MenuPage' already exists.


# **Integrity Constraint Violations in Original Dataset**

In [ ]:
# ICV 1: Check integrity constraints for ‘id’, cannot be NULL in Menu, MenuPage, MenuItem and Dish (1)
query = '''SELECT id FROM Menu
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [ ]:
# ICV 1: (2)
query = '''SELECT id FROM Dish
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [ ]:
# ICV 1: (3)
query = '''SELECT id FROM MenuItem
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [ ]:
# ICV 1: (4)
query = '''SELECT id FROM MenuPage
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [ ]:
# ICV 2: Check integrity constraints for ‘id’, cannot have duplicates in Menu, MenuPage, MenuItem and Dish (1)
query = '''SELECT id, COUNT(*) FROM Menu
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [ ]:
# ICV 2: (2)
query = '''SELECT id, COUNT(*) FROM Dish
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [ ]:
# ICV 2: (3)
query = '''SELECT id, COUNT(*) FROM MenuItem
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [ ]:
# ICV 2: (4)
query = '''SELECT id, COUNT(*) FROM MenuPage
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [ ]:
# ICV 3: Check the integrity constraints for the ‘menus_appeared’, cannot be NULL and cannot be negative, in Dish.csv
query = '''SELECT id, menus_appeared FROM Dish
            WHERE menus_appeared IS NULL OR menus_appeared < 0'''
pd.read_sql_query(query, conn)

,id,menus_appeared


In [ ]:
# ICV 4: Check the data quality constraint , if the ‘lowest_price’ is less than or equal to the ‘highest_price’ in Dish
query = '''SELECT id, lowest_price, highest_price FROM Dish
           WHERE lowest_price > highest_price'''
pd.read_sql_query(query, conn)

,id,lowest_price,highest_price


In [ ]:
# ICV 5: Check if the data quality constraint, ‘first_appeared’ should be less than or equal to ‘last_appeared’ for a Dish
query = '''SELECT id, first_appeared, last_appeared FROM Dish
           WHERE first_appeared > last_appeared'''
pd.read_sql_query(query, conn)

,id,first_appeared,last_appeared
0,164029,1900,0
1,204888,1900,0
2,250693,1945,0
3,250699,1945,0
4,301736,1940,0
5,309629,1947,0


In [ ]:
# ICV 6: Check if there are any violations for the constraint, ‘page_count’ in the Menu.csv cannot be 0 or NULL
query = '''SELECT * FROM Menu
           WHERE page_count = 0 OR page_count IS NULL
        '''
pd.read_sql_query(query, conn)

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count


In [ ]:
# ICV 7: Check if there are any foreign key constraint violations for ‘menu_id’ in MenuPage.csv which should yield a
# valid id, ‘id’ in Menu.csv
query = '''SELECT DISTINCT menu_id FROM MenuPage
           WHERE menu_id NOT IN (
              SELECT id FROM Menu
           )
           '''
pd.read_sql_query(query, conn)

,menu_id
0,12460
1,12461
2,12462
3,12544
4,12566
...,...
2266,35521
2267,35522
2268,35523
2269,35524


In [9]:
# ICV 8: Check that 'venue' is not NULL in Menu
query = '''SELECT venue FROM Menu
           WHERE venue IS NULL
           '''
pd.read_sql_query(query, conn)

,venue
0,None
1,None
2,None
3,None
4,None
...,...
9421,None
9422,None
9423,None
9424,None


In [11]:
# ICV 9: Check that 'event' is not NULL in Menu
query = '''SELECT event FROM Menu
           WHERE event IS NULL
           '''
pd.read_sql_query(query, conn)

,event
0,None
1,None
2,None
3,None
4,None
...,...
9386,None
9387,None
9388,None
9389,None


In [9]:
# ICV 10: Check that 'name' is not NULL in Dish
query = '''SELECT name FROM Dish
           WHERE name IS NULL
           '''
pd.read_sql_query(query, conn)

,name


# **Integrity Constraint Violations in Cleaned Dataset**

In [6]:
menu_cleaned_csv = '/content/drive/My Drive/CS513/Menu_cleaned.csv'
dish_cleaned_csv = '/content/drive/My Drive/CS513/Dish_cleaned.csv'
menuitem_cleaned_csv = '/content/drive/My Drive/CS513/Menu_item_cleaned.csv'
menu_join_menu_page_csv = '/content/drive/My Drive/CS513/menu_join_menu_page.csv'

df = pd.read_csv(menu_cleaned_csv)
try:
  df.to_sql('MenuCleaned', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(dish_cleaned_csv)
try:
  df.to_sql('DishCleaned', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(menuitem_cleaned_csv)
try:
  df.to_sql('MenuItemCleaned', conn, index=False)
except Exception as e:
  print(e)
df = pd.read_csv(menu_join_menu_page_csv)
try:
  df.to_sql('MenuJoinMenuPage', conn, index=False)
except Exception as e:
  print(e)

Table 'MenuCleaned' already exists.
Table 'DishCleaned' already exists.
Table 'MenuItemCleaned' already exists.
Table 'MenuJoinMenuPage' already exists.


In [14]:
# ICV 1: Check integrity constraints for ‘id’, cannot be NULL in MenuCleaned, MenuJoinMenuPage (‘menu_page_id’), MenuItemCleaned and DishCleaned (1)
query = '''SELECT id FROM MenuCleaned
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [15]:
# ICV 1: (2)
query = '''SELECT id FROM DishCleaned
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [16]:
# ICV 1: (3)
query = '''SELECT id FROM MenuItemCleaned
            WHERE id IS NULL'''
pd.read_sql_query(query, conn)

,id


In [7]:
# ICV 1: (4)
query = '''SELECT menu_page_id FROM MenuJoinMenuPage
           WHERE menu_page_id IS NULL
            '''
pd.read_sql_query(query, conn)

,menu_page_id


In [18]:
# ICV 2: Check integrity constraints for ‘id’, cannot have duplicates in MenuCleaned, MenuJoinMenuPage (‘menu_page_id’), MenuItemCleaned and DishCleaned (1)
query = '''SELECT id, COUNT(*) FROM MenuCleaned
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [19]:
# ICV 2: (2)
query = '''SELECT id, COUNT(*) FROM DishCleaned
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [20]:
# ICV 2: (3)
query = '''SELECT id, COUNT(*) FROM MenuItemCleaned
            GROUP BY id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,id,COUNT(*)


In [21]:
# ICV 2: (4)
query = '''SELECT menu_page_id, COUNT(*) FROM MenuJoinMenuPage
            GROUP BY menu_page_id
            HAVING COUNT(*) > 1'''
pd.read_sql_query(query, conn)

,menu_page_id,COUNT(*)


In [22]:
# ICV 3: Check the integrity constraints for the ‘menus_appeared’, cannot be NULL and cannot be negative, in DishCleaned
query = '''SELECT id, menus_appeared FROM DishCleaned
            WHERE menus_appeared IS NULL OR menus_appeared < 0'''
pd.read_sql_query(query, conn)

,id,menus_appeared


In [23]:
# ICV 4: Check the data quality constraint , if the ‘lowest_price’ is less than or equal to the ‘highest_price’ in Dish
query = '''SELECT id, lowest_price, highest_price FROM DishCleaned
           WHERE lowest_price > highest_price'''
pd.read_sql_query(query, conn)

,id,lowest_price,highest_price


In [24]:
# ICV 5: Check if the data quality constraint, ‘first_appeared’ should be less than or equal to ‘last_appeared’ for a Dish
query = '''SELECT id, first_appeared, last_appeared FROM DishCleaned
           WHERE first_appeared > last_appeared'''
pd.read_sql_query(query, conn)

,id,first_appeared,last_appeared


In [8]:
# ICV 6: Check if there are any violations for the constraint, ‘page_count’ in the MenuCleaned cannot be 0 or NULL
query = '''SELECT * FROM MenuCleaned
           WHERE page_count = 0 OR page_count IS NULL
        '''
pd.read_sql_query(query, conn)

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,date,location,currency,currency_symbol,status,page_count,dish_count


In [8]:
# ICV 7: Check if there are any foreign key constraint violations for ‘menu_id’ in MenuJoinMenuPage which should yield a
# valid id, ‘id’ in MenuCleaned
query = '''SELECT DISTINCT menu_id FROM MenuJoinMenuPage
           WHERE menu_id NOT IN (
              SELECT id FROM MenuCleaned
           )
           '''
pd.read_sql_query(query, conn)

,menu_id


In [12]:
# ICV 8: Check that 'venue' is not NULL in MenuCleaned
query = '''SELECT venue FROM MenuCleaned
           WHERE venue IS NULL
           '''
pd.read_sql_query(query, conn)

,venue


In [13]:
# ICV 9: Check that 'event' is not NULL in MenuCleaned
query = '''SELECT event FROM MenuCleaned
           WHERE event IS NULL
           '''
pd.read_sql_query(query, conn)

,event
